In [1]:
# Reference : https://github.com/Hvass-Labs/TensorFlow-Tutorials

#import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import pickle
import prettytensor as pt
import time

%matplotlib inline

In [2]:
# Load CIFAR10
cifar_dir = "./data/CIFAR-10/cifar-10-batches-py/"
cifar_train_cache = "./data/CIFAR-10/inception_cifar10_train.pkl"
cifar_test_cache = "./data/CIFAR-10/inception_cifar10_test.pkl"
train_sz = 50000
test_sz = 10000
img_sz = 32
num_channels = 3
num_classes = 10


In [3]:
def unpickle(filename):
    fpath = cifar_dir+filename
    print("Loading "+fpath)
    file = open(fpath,mode='rb')
    data = pickle.load(file, encoding='bytes')
#     data = pickle.load(file, encoding='utf-8')
    file.close()
    return data

def decode(filename):
    data = unpickle(filename)
    raw_images = data[b'data']
    cls = np.array(data[b'labels'])
    images = np.array(raw_images, dtype=float).reshape([-1, num_channels, img_sz, img_sz])
    images = images.transpose([0, 2, 3, 1])
    return images, cls

def one_hotit(data):
    return np.eye(num_classes, dtype=float)[data]

def load_data():
    print("Loading tarin data")
    train_images = np.zeros(shape=[train_sz, img_sz, img_sz, num_channels], dtype=float)
    train_int_labels = np.zeros(shape=[train_sz], dtype=int)

    begin = 0
    for i in range(5):
        data_file = "data_batch_"+str(i+1)
        print("Loading "+data_file)
        images_batch, cls_batch = decode(data_file)
        num_images = len(images_batch)
        end = begin + num_images
        train_images[begin:end, :] = images_batch
        train_int_labels[begin:end] = cls_batch
        begin = end
        
    print("Loading test data")
    test_images, test_int_labels = decode("test_batch")
    return train_images, train_int_labels, test_images, test_int_labels



In [4]:
train_images, train_int_labels, test_images, test_int_labels = load_data()

Loading tarin data
Loading data_batch_1
Loading ./data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data_batch_2
Loading ./data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data_batch_3
Loading ./data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data_batch_4
Loading ./data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data_batch_5
Loading ./data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading test data
Loading ./data/CIFAR-10/cifar-10-batches-py/test_batch


In [5]:
train_labels = one_hotit(train_int_labels)
test_labels = one_hotit(test_int_labels)

In [6]:
metadata = unpickle("batches.meta")[b'label_names']
classnames = [x.decode('utf-8') for x in metadata]

Loading ./data/CIFAR-10/cifar-10-batches-py/batches.meta


In [7]:
# Load inception model
# Reference : https://github.com/Hvass-Labs/TensorFlow-Tutorials
import inception
from inception import transfer_values_cache
model = inception.Inception()

In [8]:
train_transfer_values = transfer_values_cache(cache_path=cifar_train_cache , images=train_images, model=model)

- Data loaded from cache-file: ./data/CIFAR-10/inception_cifar10_train.pkl


In [9]:
test_transfer_values = transfer_values_cache(cache_path=cifar_test_cache , images=test_images, model=model)

- Data loaded from cache-file: ./data/CIFAR-10/inception_cifar10_test.pkl


In [10]:
# TF variables
x = tf.placeholder(tf.float32, shape=[None, model.transfer_len], name='x')
y_onehot = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_onehot')
y_int = tf.argmax(y_onehot, dimension=1)


# Neural Network
x_pretty = pt.wrap(x)
with pt.defaults_scope(activation_fn=tf.nn.relu):
    y_pred, loss = x_pretty.\
    fully_connected(size=100, name='layer_fc1').\
    fully_connected(size=10, name='layer_fc2').\
    softmax_classifier(num_classes=num_classes, labels=y_onehot)

# Optimizer and step counter
global_step = tf.Variable(initial_value=0, name='global_step', trainable=False)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss, global_step)
    
# TF test 
ty_int = tf.argmax(y_pred, dimension=1)
correct_prediction = tf.equal(y_int, ty_int)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
session = tf.Session()

In [12]:
session.run(tf.global_variables_initializer())

In [13]:
batch_size = 256 # Change to limit RAM consumption

In [14]:
def random_batch():
    idx = np.random.choice(train_sz, size=batch_size, replace=False)
    x_batch = train_transfer_values[idx]
    y_batch = train_labels[idx]
    return x_batch, y_batch

In [15]:
def optimize(num_iterations):
    tstart = time.time()
    for i in range(num_iterations):
        x_batch, y_batch = random_batch()
        feed_dict_train = {x: x_batch, y_onehot: y_batch}
        i_global,_ = session.run([global_step, optimizer], feed_dict=feed_dict_train)
        if (i_global % 50 == 0) or (i == num_iterations - 1):
            batch_acc = session.run(accuracy, feed_dict=feed_dict_train)
            msg = "Global Step: {0:>6}, Training Batch Accuracy: {1:>6.1%}"
            print(msg.format(i_global, batch_acc))

    tend = time.time()
    diff = tend - tstart
    print("Total time : " + str(diff) + " seconds")

In [16]:
def predict():
    pred = np.zeros(shape=test_sz, dtype=np.int)
    i = 0
    while i < test_sz:
        j = min(i + batch_size, test_sz)
        feed_dict = {x: test_transfer_values[i:j], y_onehot: test_labels[i:j]}
        pred[i:j] = session.run(ty_int, feed_dict=feed_dict)
        i = j
    correct = (test_int_labels == pred)
    return correct, pred

In [17]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(cls_pred):
    cm = confusion_matrix(y_true=test_int_labels, y_pred=cls_pred)
    for i in range(num_classes):
        # Append the class-name to each line.
        classname = "({}) {}".format(i, classnames[i])
        print(cm[i, :], classname)
    # Print the class-numbers for easy reference.
    class_numbers = [" ({0})".format(i) for i in range(num_classes)]
    print("".join(class_numbers))

In [18]:
def print_accuracy():
    correct, pred = predict()
    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(correct.mean(), correct.sum(), test_sz))
    print("Confusion Matrix:")
    plot_confusion_matrix(pred)

In [19]:
print_accuracy()

Accuracy on Test-Set: 9.3% (933 / 10000)
Confusion Matrix:
[ 25 518   0 200   1   9 117   0  31  99] (0) airplane
[ 41  83   0  53   4   7 185   0  82 545] (1) automobile
[ 64 552   0 121   4   6 133   0  23  97] (2) bird
[ 43 324   0 161   3   2 165   0  25 277] (3) cat
[ 56 586   0  44   1   3 178   0   7 125] (4) deer
[ 60 349   0 222  10   4 115   0   5 235] (5) dog
[149 487   0  80   4  19  67   0  13 181] (6) frog
[ 53 678   0  57   1   3 123   0  17  68] (7) horse
[ 39 653   0  74   3   0 125   0  65  41] (8) ship
[ 14 252   0  76   0  11 115   0   5 527] (9) truck
 (0) (1) (2) (3) (4) (5) (6) (7) (8) (9)


In [20]:
optimize(num_iterations=100)

Global Step:     50, Training Batch Accuracy:  39.1%
Global Step:    100, Training Batch Accuracy:  57.0%
Total time : 1.2875735759735107 seconds


In [21]:
print_accuracy()

Accuracy on Test-Set: 57.1% (5709 / 10000)
Confusion Matrix:
[839   0   5   7   0  12   0   0 105  32] (0) airplane
[419 177   3   5   0 134   0   0 126 136] (1) automobile
[216   0 421  54   4  41  96   0 145  23] (2) bird
[ 64   0   1 516   0 163 135   0  12 109] (3) cat
[ 98   0   9   9 214 104  91   0  52 423] (4) deer
[ 49   0   1  58   1 790  21   0   1  79] (5) dog
[ 17   0   5  14   0  32 911   0  10  11] (6) frog
[211   0  18  64  12  50   5   0 295 345] (7) horse
[ 59   1   0   1   0   1   1   0 926  11] (8) ship
[ 45   2   0   4   0   3   1   0  30 915] (9) truck
 (0) (1) (2) (3) (4) (5) (6) (7) (8) (9)
